# Декораторы

Декоратор это функция, которая в качестве одного из аргументов принимает объект и что-то возвращает. Декораторы в Python можно применять ко всему: функциям, классам и методам. Основная цель декораторов – изменить поведение объекта, не меняя сам объект. Это очень гибкая функциональная возможность языка.

Декорирование функций происходит с помощью следующего синтаксиса

```Python
@decorator
def function():
    ...
```

Такая запись будет аналогично следующему определению:

```Python
def function():
    ...

function = decorator(function)
```

В этом случае результат выполнения функции ```decorator``` записывается обратно по имени ```function```. Таким образом происходит подмена функции, связанной с именем ```function```, на результат выполнения декоратора.

С помощью декораторов можно, например, измерять время выполнения функций, контролировать количество вызовов, кеширование, вывод предупреждений об использовании устаревших функций, трассировка, использование в контрактном программировании.

Рассмотрим пример измерения времени выполнения кода функции.

In [5]:
import time

def timeit(f):
    """Декоратор вычисления времени выполнения функции"""
    def inner(*args, **kwargs):
        start = time.time()
        # запускаем декорируемую функцию, передавая ее параметры
        # имя f определено в объемлющей области видимости
        res = f(*args, **kwargs)
        end = time.time()
        print(f'{end - start} seconds')
        # не забываем вернуть результат выполнения исходной функции
        return res
    return inner


@timeit
def my_sum(*args, **kwargs):
    """Функция суммы"""
    return sum(*args, **kwargs)


res = my_sum([i for i in range(int(1e5))])

0.0010006427764892578 seconds


В такой реализации есть несколько проблем:
- нет возможности отключить трассировку;
- вывод в стандартный поток вывода (```sys.stdout```);
- пропала строка документации и атрибуты декорируемой функции.

In [6]:
print(f'{my_sum.__name__ = }')
print(f'{my_sum.__doc__ = }')
help(my_sum)

my_sum.__name__ = 'inner'
my_sum.__doc__ = None
Help on function inner in module __main__:

inner(*args, **kwargs)



Так как в Python функции являются объектами, то их можно изменять во время выполнения. В этом кроется решение упомянутых выше проблем. Можно скопировать нужные атрибуты декорируемой функции.

Чтобы не копировать каждый атрибут вручную существует готовая реализация этого функционала в модуле ```functools``` стандартной библиотеки.

In [7]:
from functools import wraps


def timeit(f):
    """Декоратор вычисления времени выполнения функции"""
    # декорируем внутреннюю функцию декоратором wraps
    # он заменит атрибуты функции inner атрибутами исходной функции
    @wraps(f)
    def inner(*args, **kwargs):
        start = time.time()
        res = f(*args, **kwargs)
        end = time.time()
        print(f'{end - start} seconds')
        return res
    return inner


@timeit
def my_sum(*args, **kwargs):
    """Функция суммы"""
    return sum(*args, **kwargs)


print(f'{my_sum.__name__ = }')
print(f'{my_sum.__doc__ = }')
help(my_sum)

my_sum.__name__ = 'my_sum'
my_sum.__doc__ = 'Функция суммы'
Help on function my_sum in module __main__:

my_sum(*args, **kwargs)
    Функция суммы



# Параметризованные декораторы

У реализованного нами декоратора сильно ограниченное применение, попробуем его расширить.

Отключение декоратора можно реализовать, используя глобальную переменную, например, ```DEC_ENABLED```, принимающую значение ```True```, если декоратор активен и ```False``` в противном случае.

Возможность вывода не только в стандартный поток (```sys.stdout```), но и в поток ошибок (```sys.stderr```) или файл можно с помощью передачи аргументов. Добавление аргументов к декораторам немного усложняет задачу.

```python
@decorator(arg)
def foo():
    ...
```

В этом случае добавляется дополнительный этап, а именно вычисление декоратора.

```python
def foo():
    ...

dec = decorator(x)  # новый этап
foo = dec(foo)
```

Решить проблему передачи аргументов можно несколькими способами. Первый из них, и не самый лучший заключается в добавлении еще одной вложенной функции.

In [8]:
import sys


DEC_ENABLED = True


def timeit(file):
    # Функция timeit теперь не является самим декоратором,
    # а только вычисляет (возвращает) его.
    # Это нужно для создания еще одного уровня объемлющей области
    # видимости для хранения алгументов декоратора.
    def dec(func):
        # Далее все идет как обычно
        @wraps(func)
        def inner(*args, **kwargs):
            start = time.time()
            res = func(*args, **kwargs)
            end = time.time()
            # Здесь можно использовать аргументы
            # из объемлющей области видимости
            print(f'{end - start} seconds', file=file)
            return res
        return inner if DEC_ENABLED else func
    return dec


@timeit(sys.stderr)
def my_sum(*args, **kwargs):
    """Функция суммы"""
    return sum(*args, **kwargs)


res = my_sum([i for i in range(int(1e5))])
print(res)

4999950000


0.0009989738464355469 seconds


Такой вариант будет работать при декорировании следующим образом ```@timeit(sys.stderr)```. Однако постоянно писать декораторы с тройной вложенностью это не путь питониста. Можно один раз сделать декоратор для декоратора, позволяющий передавать аргументы (да, декоратор для декоратора).

In [9]:
from functools import update_wrapper


def with_args(dec):
    """Декоратор позволяющий передавать аргументы в декоратор"""
    @wraps(dec)
    def wrapper(*args, **kwargs):
        # wrapper это обертка, принимающая аргументы декоратора
        def decorator(func):
            res = dec(func, *args, **kwargs)
            update_wrapper(res, func)
            return res
        return decorator
    return wrapper

Функция ```with_args``` принимает декоратор, оборачивает его в обертку ```wrapper```, внутри которой происходит создание нового декоратора. Исходный декоратор при этом не изменяется.

In [10]:
DEC_ENABLED = True


# теперь можно указать аргументы сразу после обязательного
# аргумента-функции func
@with_args
def timeit(func, file):
    def inner(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time()
        print(f'{end - start} seconds', file=file)
        return res
    return inner if DEC_ENABLED else func


@timeit(sys.stderr)
def my_sum(*args, **kwargs):
    """Функция суммы"""
    return sum(*args, **kwargs)


res = my_sum([i for i in range(int(1e5))])
print(res)

4999950000


0.0020020008087158203 seconds


Однако это все еще слишком сложно. Гораздо удобнее добавить возможность
вызывать декоратор без аргументов, т.е. добавить два варианта работы:

```python
@dec
def func():
    ...
```

```python
@dec(a=1, b=2, с=3)
def func():
    ...
```

Попробуем воспользоваться только ключевыми аргументами.

In [12]:
DEC_ENABLED = True


def timeit(func=None, *, file=sys.stderr):
    # Здесь используем * для определения file как ключевого аргумента
    # дополнительно задаем аргументу func значение поумолчанию.
    # Это необходимо для определения варианта вызова 
    # Условие ниже выполнится только в случае вызова декоратора с аргументами
    if func is None:
        def dec(func):
            return timeit(func, file=file)
        return dec if DEC_ENABLED else func
    @wraps(func)
    def inner(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time()
        print(f'{end - start} seconds', file=file)
        return res
    return inner if DEC_ENABLED else func

Теперь декоратор ```timeit``` можно вызывать двумя способами. Во-первых, не передавая никаких аргументов. Тогда вывод будет осуществляться в стандартный поток вывода. При этом помня, что декоратор раскрывается как ```f = timeit(f)```, можно видеть, что аргумент ```func``` принимает значение функции ```f```. Тогда первое условие не будет выполнено, а будет создана обертка ```inner```.

In [13]:
DEC_ENABLED = True


@timeit
def my_sum(*args, **kwargs):
    """Функция суммы"""
    return sum(*args, **kwargs)


res = my_sum([i for i in range(int(1e5))])
print(res)

4999950000


0.0019986629486083984 seconds


Во-вторых, передавая в качестве именованного аргумента ```file``` ```sys.stderr``` имя файла. В этом случае происходит явный вызов декоратора ```timeit(file=sys.stderr)``` без передачи аргумента ```func```, в связи с этим он принимает значение ```None```, а значит, выполняется первое условие и создается обертка ```dec```. Стоит отметить, что этот способ реализации декоратора позволяет передавать только ключевые аргумента. В большинстве случаев это улучшает читаемость кода, т.к. сразу видно за что отвечают определенные значения. 

In [14]:
DEC_ENABLED = True


@timeit(file=sys.stderr)
def my_sum(*args, **kwargs):
    """Функция суммы"""
    return sum(*args, **kwargs)


res = my_sum([i for i in range(int(1e5))])
print(res)

4999950000


0.0009996891021728516 seconds


Благодаря переменной ```DEC_ENABLED``` измерение времени можно отключить. В этом случае никаких накладных расходов, связанных с вызовом дополнительных функций не будет.

К одной функции можно применить сразу несколько декораторов, порядок их работы будет зависеть от порядка их применения к функции. Рассмотрим на примере гамбургера.

In [15]:
def with_bun(f):
    @wraps(f)
    def inner():
        print('-' * 8)
        f()
        print('-' * 8)
    return inner


def with_vegetables(f):
    @wraps(f)
    def inner():
        print(' onion')
        f()
        print(' tomato')
    return inner


def with_sauce(f):
    @wraps(f)
    def inner():
        print(' sauce')
        f()
    return inner

Определим основную функцию и задекорируем ее.

In [16]:
@with_bun
@with_vegetables
@with_sauce
def burger():
    print(' cutlet')


burger()

--------
 onion
 sauce
 cutlet
 tomato
--------


Если записать явно такое декорирование, то получиться следующая последовательность вызовов:

In [17]:
def burger():
    print(' cutlet')


burger = with_sauce(burger)
burger = with_vegetables(burger)
burger = with_bun(burger)


burger()

--------
 onion
 sauce
 cutlet
 tomato
--------


Первым будет применяться самый нижний (внутренний) декоратор. Если
изменить последовательность декорирования, то результат ожидаемо
изменится.

Вот еще пару примеров декораторов. Декоратор трассировки вызовов функций:

In [20]:
def trace(function=None, *, file=sys.stderr):
    if function is None:
        def dec(function):
            return trace(function, file=file)
        return dec if DEC_ENABLED else function

    @wraps(function)
    def inner(*args, **kwargs):
        print(f'{function.__name__}, {args}, {kwargs}')
        return function(*args, **kwargs)
    return inner if DEC_ENABLED else function


@trace
def foo():
    print('Nothing')


foo()
print('-' * 25)


@trace
def bar():
    foo()
    print('Unagi')


bar()

foo, (), {}
Nothing
-------------------------
bar, (), {}
foo, (), {}
Nothing
Unagi


Декоратор проверки входа пользователя в систему (в упрощенном виде). 

In [45]:
def is_authenticated(user):
    return user in ('monty', 'guido')


def login_required(function=None, login_url=''):
    def user_passes_test(view_func):
        @wraps(view_func)
        def wrapped(user, *args, **kwargs):
            if is_authenticated(user):
                return view_func(user, *args, **kwargs)
            print(f'Пользователь {user} перенаправлен на страницу логина: {login_url}')
        return wrapped

    if function:
        return user_passes_test(function)
    return user_passes_test


@login_required(login_url='localhost/login')
def foo(user):
    print(f'{user = }')


foo('monty')
foo('guido')
foo('pyuty')

user = 'monty'
user = 'guido'
Пользователь pyuty перенаправлен на страницу логина: localhost/login


# Полезные ссылки

- [Decorators with parameters?](https://stackoverflow.com/questions/5929107/decorators-with-parameters)
- [Reuven M. Lerner - Practical decorators - PyCon 2019](https://www.youtube.com/watch?v=MjHpMCIvwsY&feature=youtu.be)
- [Добавление атрибута к функции](https://stackoverflow.com/questions/47056059/best-way-to-add-attributes-to-a-python-function)